In [4]:
import cv2
import time
import numpy as np
import pygame
import mediapipe as mp
from scipy.spatial import distance as dist
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
from twilio.rest import Client
import os
account_sid = "AC4676f34b2a25a53d31491e5d495390e6"
auth_token = "3027cfaf50b23364a2e59ad2f1fd36bd"
client = Client(account_sid,auth_token)

Allonge = False

#def smsFunction(c = client):
 #            c.messages.create(to = ["+213555150793"],
  #                             from_ = "+18507711254",
   #                            body = 'IMEGENCY\n a male 26 years old unconcious at altius les vergies bir mourad rais \n Previously diagnosed with diabetes')

pTime = 0
cpt = 0
thresh_cons_FRAMES = 40

# For webcam input:
cap = cv2.VideoCapture(1)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1300)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 750)

with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
    while cap.isOpened() and not Allonge:
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        # Draw the pose annotation on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        if results.pose_landmarks:
            keypoints = []
            for data_point in results.pose_landmarks.landmark:
                ih, iw, ic = image.shape
                x,y,z=int(data_point.x*iw),int(data_point.y*ih),int(data_point.z*100)
                keypoints.append([x,y,z])
                #cv2.putText(image,str(data_point.x)+','+str(data_point.y),(x,y),cv2.FONT_HERSHEY_COMPLEX,.8,(0,255,0),2)
            foot_Y = (keypoints[29][1]+keypoints[30][1])/2
            h=foot_Y-keypoints[0][1]
            
            if h<=100 :
                cpt += 1
                cv2.putText(image,str(cpt),(200,200),cv2.FONT_HERSHEY_COMPLEX,.8,(0,255,0),2)
                if cpt>thresh_cons_FRAMES:
                    Allonge = True
            elif h>100:
                cpt = 0
                
            cv2.putText(image,str(h),(50,100),cv2.FONT_HERSHEY_COMPLEX,.8,(0,255,0),2)
        # Calcul fps
        cTime = time.time()
        fps = 1/(cTime-pTime)
        thresh_cons_FRAMES = fps * 6
        pTime = cTime    
        cv2.putText(image, f'FPS: {int(fps)}',(20,70),cv2.FONT_HERSHEY_PLAIN,3,(0,255,0),3)
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Pose', image)
        cpt += 1
        if cv2.waitKey(1) == 27 :
            break
        
cap.release()
cv2.destroyAllWindows()
Message = True
pTime = 0
cpt = 0
thresh_cons_FRAMES = 40
Okay = True
pygame.init()
cap = cv2.VideoCapture(1)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1300)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 750)
with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    while cap.isOpened() and Message:
        success, image = cap.read()
        display=image
        
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)
        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        cpt += 1
        if cpt == 1:
            pygame.mixer.Sound("AreUOkay.wav").play()
        cv2.putText(image,"ARE YOU OKAY ?!",(100,100),cv2.FONT_HERSHEY_COMPLEX,.8,(0,0,255),2)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())
                hand=[]
                for id, lm in enumerate(hand_landmarks.landmark):
                    ih, iw, ic = image.shape
                    x,y,z = int(lm.x*iw), int(lm.y*ih), int(lm.z*100)
                    #cv2.putText(image, str(id), (x,y),cv2.FONT_HERSHEY_PLAIN,0.8, (0,255,0), 1)
                    hand.append([x,y,z])
                
                dist_hand = dist.euclidean(hand[4],hand[8])/dist.euclidean(hand[7],hand[6])
                
                if dist_hand < 1 and cpt<thresh_cons_FRAMES:
                    cv2.putText(image, 'OK: ', 
                                    (10,200),cv2.FONT_HERSHEY_PLAIN,
                                    1.5, (0,255,0),1)
                    Message = False
                
                elif cpt>thresh_cons_FRAMES:
                    cv2.putText(image,"Danger !",(100,100),cv2.FONT_HERSHEY_COMPLEX,.8,(0,0,255),2)
                    #smsFunction(client)
        # Calcul fps
        cTime = time.time()
        fps = 1/(cTime-pTime)
        thresh_cons_FRAMES = fps * 15
        pTime = cTime    
        cv2.putText(image, f'FPS: {int(fps)}',(20,70),cv2.FONT_HERSHEY_PLAIN,3,(0,255,0),3)      
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Hands',image)
        if cv2.waitKey(1) == 27:
            break
cap.release()
cv2.destroyAllWindows()